In [1]:
def traceray(self, direction, spheres = None, lights = None):
    origin = self.position
    min_t = self.min_t
    max_t = self.max_t
    spheres = spheres

    closest_t = math.inf
    closest_sphere = None

    for i,sphere in enumerate(spheres):
        ts = self.intersect_ray_sphere(direction, sphere)
        if ts[0] < closest_t and min_t < ts[0] and ts[0] < max_t:
            closest_t = ts[0]
            closest_sphere = sphere
        if ts[1] < closest_t and min_t < ts[1] and ts[1] < max_t:
            closest_t = ts[1]
            closest_sphere = sphere

    if closest_sphere == None:
        return self.background_color

기존 traceray 코드에서 closest intersection 코드를 함수로 구현하였다.    
computelighting 에서 shadow를 확인하기 위해 closest sphere을 구해야 하기 때문이다.   

In [3]:
def ClosestIntersection(self, origin, direction, t_min, t_max, spheres):
        closet_t = math.inf
        closet_sphere = None
        
        for sphere in spheres:
            
            t = self.intersect_ray_sphere(origin, direction, sphere)
            
            if (t_min < t[0] < t_max) & (t[0] < closet_t):
                closet_t = t[0]
                closet_sphere = sphere
                
            if (t_min < t[1] < t_max) & (t[1] < closet_t):
                closet_t = t[1]
                closet_sphere = sphere
                
        return closet_sphere, closet_t 
    

def traceray(self,
             origin, 
             direction,
             spheres = None, 
             lights = None,
             recursion_depth=3):
    origin = origin
    t_min = self.t_min
    t_max = self.t_max
    spheres = spheres
    closest_sphere, closest_t = self.ClosestIntersection(origin,
                                                    direction,
                                                    t_min,
                                                    t_max,
                                                    spheres)

    if closest_sphere == None:
        return self.background_color

    point = np.add(origin, np.multiply(closest_t, direction))
    normal = np.subtract(point, closest_sphere.center)
    normal = np.multiply(1.0 / np.linalg.norm(normal), normal)
    view = np.multiply(-1, direction)

    local_color = self.ComputeLighting(point, normal, view, closest_sphere.specular, lights, spheres) * closest_sphere.color


In [ ]:
def ComputeLighting(self, point, normal, view, specular, lights, spheres):
    intensity = 0
    length_n = np.linalg.norm(normal)
    length_v = np.linalg.norm(view)
    for i, light in enumerate(lights):
        if light.ltype == 0:
            intensity += light.intensity
        else :
            vec_l = []
            if light.ltype == 1:
                vec_l = np.subtract(light.position, point)
                t_max = 1
            else :  #DIRECTIONAL
                vec_l = light.direction
                t_max = math.inf

       # Shadow check
            shadow_sphere, shadow_t = self.ClosestIntersection(point, vec_l, 0.001, t_max, spheres)
            if shadow_sphere != None:
                continue

      # Diffuse reflection.
            n_dot_l = np.dot(normal, vec_l)
            if( n_dot_l > 0) :
                intensity += light.intensity * n_dot_l / (length_n * np.linalg.norm(vec_l))


      # Specular reflection.
            if (specular != -1 ):
                vec_r = 2 * normal * np.dot(normal, vec_l) - vec_l
                # vec_r = np.subtract(np.multiply(2.0*np.dot(normal, vec_l), normal), vec_l)
                r_dot_v = np.dot(vec_r, view)
                if (r_dot_v > 0):
                    intensity += light.intensity * math.pow(r_dot_v / (np.linalg.norm(vec_r) * length_v), specular)
    return np.array(intensity)